In [2]:
import matplotlib.pyplot as plt
import numpy as np
import keras


from sklearn.model_selection import cross_validate

from evaluator import Plotter, Evaluator
from dataset_preprocessor import Preprocessor, MinMax_Scaler, Standard_Scaler
from database import Database

In [3]:
db_train = Database('dataset_train')
db_train.print_balance()
db_val = Database('Dataset_validation')
db_val.print_balance()
db_test = Database('Dataset_test')
db_test.print_balance()

positives_no_aug: 2311
negatives_no_aug: 14040
postives_aug: 9264
negatives_aug: 56264
Total_positives: 11575
Total_negatives: 70304
positives_no_aug: 764
negatives_no_aug: 4686
postives_aug: 0
negatives_aug: 0
Total_positives: 764
Total_negatives: 4686
positives_no_aug: 770
negatives_no_aug: 4680
postives_aug: 0
negatives_aug: 0
Total_positives: 770
Total_negatives: 4680


In [10]:
(
    
    _,
    _,
    x_train,
    y_train,
    _,
    _,
) = db_train.get_datasets()

(
    x_test,
    y_test,
    _,
    _,
    _,
    _,
) = db_test.get_datasets()

(
    x_val,
    y_val,
    _,
    _,
    _,
    _,
) = db_val.get_datasets()


In [17]:
import re
import pandas as pd

In [29]:
rgx = re.compile('.*Enterface.*')
enterface = pd.DataFrame(list(db_train.collection.find({'_id':rgx, 'augmented':False})))
y_train = enterface["label"].to_numpy()
x_train = enterface.drop(columns=["_id", "augmented", "label"]).to_numpy()



In [30]:
scaler = MinMax_Scaler()
preprocessor = Preprocessor(scaler, 1)
x_train, y_train = preprocessor.downsample(x_train, y_train, 0, 0.73)
x_train = preprocessor.scale(x_train, y_train)
# x_train, y_train = preprocessor.shuffle(x_train, y_train)
print(len(y_train))
print(sum(y_train))

329
126


## MLP Classifier - Manual

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, AlphaDropout
from tensorflow.keras.models import Model, load_model, Sequential
from numpy.random import seed
tf.random.set_seed(2)



In [33]:
mlp = Sequential()
mlp.add(Input(shape=(57, )))
mlp.add(Dense(8, activation='relu'))
mlp.add(Dense(4, activation='relu'))
mlp.add(Dense(1, activation='softmax'))
mlp.summary()
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=tf.keras.metrics.Accuracy())
history = mlp.fit(x=x_train, y=y_train, epochs=100, validation_data=(x_test, y_test), verbose=1).history

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 8)                 464       
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 5         
Total params: 505
Trainable params: 505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 1s 70ms/step - loss: 0.6909 - accuracy: 0.3735 - val_loss: 8.7273 - val_accuracy: 0.1413
Epoch 2/100
11/11 [==============================] - 1s 59ms/step - loss: 0.6894 - accuracy: 0.3653 - val_loss: 8.5949 - val_accuracy: 0.1413
Epoch 3/100
11/11 [==============================] - 1s 58ms/step - loss: 0.6870 - accuracy: 0.3881 - 

In [34]:
train_loss=[value for key, value in history.items() if 'loss' in key.lower()][0]
valid_loss=[value for key, value in history.items() if 'loss' in key.lower()][1]
train_recall=[value for key, value in history.items() if 'recall' in key.lower()][0]
valid_recall=[value for key, value in history.items() if 'recall' in key.lower()][1]

plotter = Plotter()
plotter.plot_loss(train_loss, valid_loss)
plotter.plot_model_recall_fpr(train_recall, valid_recall)

IndexError: list index out of range

In [35]:
evaluator = Evaluator()
train, test, _ = evaluator.evaluate(mlp, x_train, y_train, x_test, y_test)

Exactitud: 0.38 / 0.14 / None
Precisión: 0.38 / 0.14 / None
Especificidad: nan / nan / None
Sensibilidad: 1.0 / 1.0 / None
true negative rate: 0.0 / None / 0.0
F - Score: 0.5507246376811594 / 0.24561403508771928 / None
